# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#weather data file
weather_data_path = "../WeatherPy/weather_data.csv"

#read the data file
weather_data_df = pd.read_csv(weather_data_path, dtype="object", encoding="utf-8")
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Contry,Date
0,albany,42.6,-73.97,57.0,89.0,100.0,7.45,US,1603163605.0
1,tlahualilo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,east london,-33.02,27.91,60.8,77.0,40.0,17.22,ZA,1603163595.0
3,khatanga,71.97,102.5,13.41,97.0,82.0,4.63,RU,1603163606.0
4,nyrob,60.73,56.72,31.35,99.0,97.0,3.76,RU,1603163606.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps w/ API key
gmaps.configure(api_key=g_key)

#store lat and lng as locations, and Humidity as the weight
locations = weather_data_df[['Lat', 'Lng']].astype(float).dropna()
humidity_w = weather_data_df['Humidity'].astype(float).dropna()

In [4]:
#Create heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_w, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Ideal weather df
ideal_weather_data = weather_data_df.loc[(weather_data_df['Max Temp'] <'80')& 
                                         (weather_data_df['Max Temp'] >'70')&
                                         (weather_data_df['Wind Speed'] < '10')&
                                         (weather_data_df['Cloudiness'] <'50')]
ideal_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Contry,Date
247,redwater,33.36,-94.25,71.6,94.0,40.0,1.77,US,1603163671.0
272,inirida,3.87,-67.92,75.85,85.0,34.0,1.39,CO,1603163677.0
405,houma,29.6,-90.72,73.99,83.0,0.0,1.99,US,1603163711.0
421,manicore,-5.81,-61.3,75.65,84.0,100.0,1.77,BR,1603163715.0
522,altagracia de orituco,9.86,-66.38,72.46,92.0,5.0,1.59,VE,1603163740.0
603,coahuayana,18.73,-103.68,79.23,87.0,41.0,1.45,MX,1603163761.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create new hotel df
hotel_df = ideal_weather_data
hotel_df.insert(0, "Hotel Name", " ")
hotel_df

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Contry,Date
247,,redwater,33.36,-94.25,71.6,94.0,40.0,1.77,US,1603163671.0
272,,inirida,3.87,-67.92,75.85,85.0,34.0,1.39,CO,1603163677.0
405,,houma,29.6,-90.72,73.99,83.0,0.0,1.99,US,1603163711.0
421,,manicore,-5.81,-61.3,75.65,84.0,100.0,1.77,BR,1603163715.0
522,,altagracia de orituco,9.86,-66.38,72.46,92.0,5.0,1.59,VE,1603163740.0
603,,coahuayana,18.73,-103.68,79.23,87.0,41.0,1.45,MX,1603163761.0


In [16]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


D:\Python\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Python\envs\PythonData\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

#spelling mistake

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
figure_layout = {
    'width':'400px',
    'height':'300px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure

#Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
#Add the layer to the map
fig.add_layer(markers)
fig

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [19]:
hotel_df


,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Contry,Date
247,,redwater,33.36,-94.25,71.6,94.0,40.0,1.77,US,1603163671.0
272,La Cabaña Guainiana,inirida,3.87,-67.92,75.85,85.0,34.0,1.39,CO,1603163677.0
405,Days Inn by Wyndham Houma LA,houma,29.6,-90.72,73.99,83.0,0.0,1.99,US,1603163711.0
421,Hotel Durval,manicore,-5.81,-61.3,75.65,84.0,100.0,1.77,BR,1603163715.0
522,Hacienda la Quinta,altagracia de orituco,9.86,-66.38,72.46,92.0,5.0,1.59,VE,1603163740.0
603,Hotel Posada Navideña,coahuayana,18.73,-103.68,79.23,87.0,41.0,1.45,MX,1603163761.0
